In [1]:
import numpy as np
#import cupy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
n_time = 8  # 時系列の数
n_in   = 2   # 入力層ニューロン数
n_mid  = 32  # 中間層ニューロン数
n_out  = 1   # 出力層ニューロン数

In [3]:
# 学習データ

max_num = 2**n_time
binaries = np.zeros((max_num, n_time), dtype=int)
for i in range(max_num):
    num10 = i
    for j in range(n_time):
        pow2 = 2 ** (n_time - 1 - j)
        binaries[i, j] = num10 // pow2
        num10 %= pow2

print(binaries)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 ...
 [1 1 1 ... 1 0 1]
 [1 1 1 ... 1 1 0]
 [1 1 1 ... 1 1 1]]


In [4]:
eta      = 0.1  # 学習係数
n_learn = 5001  # 学習回数
interval = 500  # 経過の表示間隔


class OutputLayer:
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)
        self.b = np.zeros(n)
    
    def activate_func(self, u):
        # sigmoid function
        return 1 / (1 + np.exp(-u))
    
    def diff_func(self, grad_y, y):
        # differencial sigmoid
        return grad_y * (1 - y) * y
    
    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = self.activate_func(u)
        return self.y
    
    def backward(self, x, y, t):
        delta = self.diff_func(y - t, y)
        self.grad_w = np.dot(x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T)
        return self.grad_x

    def reset_sum_grad(self):
        self.grad_w = np.zeros_like(self.w)
        self.grad_b = np.zeros_like(self.b)
    
    def update(self, eta):
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b



class RnnBaseLayer:
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)
        self.v = np.random.randn(n, n) / np.sqrt(n)
        self.b = np.zeros(n)

    # def activate_func(self, u):
    #     # simplly tanh
    #     return np.tanh(u)
    
    # def diff_func(self, grad_y, y):
    #     # differencial sigmoid
    #     return grad_y * (1 - y) * y
    
    def forward(self, x, prev_y):
        u = np.dot(x, self.w) + np.dot(prev_y, self.v) + self.b
        self.y = np.tanh(u)
        return self.y
    
    def backward(self, x, y, prev_y, grad_y):
        delta = grad_y * (1 - y**2)

        self.grad_w += np.dot(x.T, delta)
        self.grad_v += np.dot(prev_y.T, delta)
        self.grad_b += np.sum(delta, axis=0)

        self.grad_x = np.dot(delta, self.w.T)
        self.grad_prev_y = np.dot(delta, self.v.T)
        return self.grad_prev_y

    def reset_sum_grad(self):
        self.grad_w = np.zeros_like(self.w)
        self.grad_v = np.zeros_like(self.v)
        self.grad_b = np.zeros_like(self.b)

    def update(self, eta):
        self.w -= eta * self.grad_w
        self.v -= eta * self.grad_v
        self.b -= eta * self.grad_b

In [5]:
rnnLayer = RnnBaseLayer(n_in, n_mid)
outputLayer = OutputLayer(n_mid, n_out)

def train(x_mb, t_mb):
    y_rnn = np.zeros((len(x_mb), n_time+1, n_mid))
    y_out = np.zeros((len(x_mb), n_time, n_out))

    # Forward propergation
    y_prev = y_rnn[:, 0, :]
    for i in range(n_time):
        # RNN layer
        x = x_mb[:, i, :]
        y = rnnLayer.forward(x, y_prev)
        y_rnn[:, i + 1, :] = y
        y_prev = y

        # output layer
        y_out[:, i, :] = outputLayer.forward(y)
    
    # back propergation
    outputLayer.reset_sum_grad()
    rnnLayer.reset_sum_grad()
    grad_y = 0
    for i in reversed(range(n_time)):
        # output layer
        x = y_rnn[:, i+1, :]
        y = y_out[:, i, :]
        t = t_mb[:, i, :]
        grad_x_out = outputLayer.backward(x, y, t)

        # Rnn layer
        x = x_mb[:, i, :]
        y = y_rnn[:, i+1, :]
        y_prev = y_rnn[:, i, :]
        grad_y = rnnLayer.backward(x, y, y_prev, grad_y + grad_x_out)

    # update
    rnnLayer.update(eta)
    outputLayer.update(eta)
    return y_out

def get_error(y, t):
    return 1.0/2.0*np.sum(np.square(y - t))

for i in range(n_learn):
    # create random
    num1 = np.random.randint(max_num//2)
    num2 = np.random.randint(max_num//2)

    # input
    x1= binaries[num1]
    x2= binaries[num2]
    x_in = np.zeros((1, n_time, n_in))
    x_in[0, :, 0] = x1
    x_in[0, :, 1] = x2
    x_in  = np.flip(x_in, axis=1)

    # create correct
    t = binaries[num1+num2]
    t_in = t.reshape(1, n_time, n_out)
    t_in = np.flip(t_in , axis=1)

    # learning
    y_out = train(x_in, t_in)
    y = np.flip(y_out, axis=1).reshape(-1)

    error = get_error(y_out, t_in)

    if i % interval == 0:
        y2 = np.where(y<0.5, 0, 1)
        y10 = 0
        for j in range(len(y)):
            pow2 = 2 ** (n_time-1-j)
            y10 += y2[j] * pow2

        print("learn count:", i)
        print("error rate:", error)
        print("output :", y2)
        print("correct:", t)

        c = "Success : " if (y2 == t).all() else "Failure : "
        print(c + str(num1) + " + " + str(num2) + " = " + str(y10))
        print("-- -- -- -- -- -- -- -- -- -- -- -- -- -- --")

n_learn: 0
error: 0.9337541878325495
output : [0 1 0 0 0 0 1 0]
correct: [0 1 0 0 0 0 0 1]
orz : 33 + 32 = 66
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 500
error: 0.6338173844864895
output : [1 1 1 1 0 1 1 1]
correct: [1 1 1 0 0 1 0 1]
orz : 107 + 122 = 247
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 1000
error: 0.08709147697811526
output : [1 0 1 0 1 1 0 1]
correct: [1 0 1 0 1 1 0 1]
\(^_^)/ : 123 + 50 = 173
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 1500
error: 0.04121356644880319
output : [0 0 1 0 1 0 0 1]
correct: [0 0 1 0 1 0 0 1]
\(^_^)/ : 18 + 23 = 41
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 2000
error: 0.018843175140588423
output : [1 1 0 0 0 0 1 1]
correct: [1 1 0 0 0 0 1 1]
\(^_^)/ : 114 + 81 = 195
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 2500
error: 0.014113131401947657
output : [0 1 1 1 1 0 1 0]
correct: [0 1 1 1 1 0 1 0]
\(^_^)/ : 93 + 29 = 122
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 3000
error: 0.021